# Finding The Topics in HathiTrust Data 

In [1]:
from htrc_features import FeatureReader
import glob 
# import modules reused from the creation of the topic model
from pool_processing_test import STOPWORDS
from pool_processing_test import PUNCDIG_TRANSLATOR
from pool_processing_test import WordNetLemmatizer 

In [ ]:
lemmatizer = WordNetLemmatizer()

In [ ]:
paths = glob.glob('../data/testfiles/*.bz2')

In [ ]:
paths

In [ ]:
fr = FeatureReader(paths)

In [ ]:
def vol_gen(feature_reader):
    for vol in feature_reader.volumes():
        yield vol

In [ ]:
volues = vol_gen(fr)

In [ ]:
vol = next(volues)

In [ ]:
vol.id

In [ ]:
vol_list = []
for vol in fr.volumes():
    for page in vol.pages():
        df = page.tokenlist('body', case=False, pos=False)
        dicty = df.to_dict()
        count = dicty['count']
        clean_list = []
        for key in count.keys():
            w = key[2]
            if w not in STOPWORDS and len(w) > 2: # removing two character words
                w = w.translate(PUNCDIG_TRANSLATOR)
                if w != '':
                    clean_list += [lemmatizer.lemmatize(w)] * count[key]
                    # clean_list.append(lemmatizer.lemmatize(w) * count[key])
        vol_list.append(clean_list)

        

In [ ]:
[lemmatizer.lemmatize("apparatus")] * 2

In [ ]:
vol_list[100]

In [ ]:
other_corpus = [corpus_dict.doc2bow(text) for text in vol_list]

In [ ]:
vector = lda_model[other_corpus[100]]

In [ ]:
vector[0]

In [ ]:
row = sorted(vector[0], key=lambda x: x[1], reverse=True)

In [ ]:
row[0][0]

In [ ]:
 for j, (topic_num, prop_topic) in enumerate(row):
        print(j, topic_num, prop_topic)

In [ ]:
pot_match = []
for doc_num, doc in enumerate(other_corpus):
    vector = lda_model[doc]
    row = sorted(vector[0], key=lambda x: x[1], reverse=True)
    topic_num, prop_topic = row[0]
    if topic_num in (0, 1, 3, 5, 6, 11):
        pot_match.append((doc_num, topic_num, prop_topic))
    
    

In [ ]:
sorted(pot_match, key=lambda x: x[-1], reverse=True)

In [ ]:
vol_list[482]

In [ ]:
# creates a list of tuples: page, topic_num, percentage
sorted_list = sorted(pot_match, key=lambda x: x[-1], reverse=True)
sorted_list[-10:]

In [ ]:
short_sort = [x for x in sorted_list if x[-1] > .04]
short_sort[-10:]

In [ ]:
print(len(short_sort))

In [ ]:
sorted_df = pd.DataFrame(sorted_list, columns=['page', 'topic_num', 'perc'])
sorted_df.groupby(['topic_num']).mean()

In [ ]:
short_sorted_df = pd.DataFrame(short_sort, columns=['page', 'topic_num', 'perc'])
short_sorted_df['mean'] = short_sorted_df.groupby('topic_num')['perc'].transform('mean')

In [ ]:
short_sorted_df['count'] = short_sorted_df.groupby(['topic_num'])['topic_num'].transform('count')

In [ ]:
short_sorted_df.groupby(['topic_num']).mean()

In [ ]:
short_sorted_df[short_sorted_df['topic_num'] == 6].plot(y='perc')

In [ ]:
from collections import Counter

In [ ]:
Counter(short_sorted_df['topic_num']).most_common()

## Running over the Corpus 

In [ ]:
from collections import namedtuple
from collections import Counter

In [ ]:
Topic = namedtuple('Topic', ['top_num', 'perc'])
BestMatch = namedtuple('BestMatch', ['page', 'top_num', 'perc'])
Book = namedtuple('Book', ['ht_id', 'top_topic', 'best_match', 'most_common_topic'])

In [ ]:
def mean(lst):
    return sum(lst) / len(lst)


def volume_parser(vol):
    vol_list = []
    for page in vol.pages():
        df = page.tokenlist('body', case=False, pos=False)
        dicty = df.to_dict()
        count = dicty['count']
        clean_list = []
        for key in count.keys():
            w = key[2]
            if w not in STOPWORDS and len(w) > 2: # removing two character words
                w = w.translate(PUNCDIG_TRANSLATOR)
                if w != '':
                    clean_list += [lemmatizer.lemmatize(w)] * count[key]
                    # clean_list.append(lemmatizer.lemmatize(w) * count[key])
        vol_list.append(clean_list)
    return vol_list


def get_topic_average(sorted_list):
    dicty = {}
    for (_, x, y) in sorted_list:
        dicty.setdefault(x, []).append(y)
    topic_averages = [(x, mean(y)) for x, y in dicty.items()]
    return topic_averages

        
def analyze_corpus_with_model(other_coprus, lda_model):
    pot_match = []
    for doc_num, doc in enumerate(other_corpus):
        vector = lda_model[doc]
        # row = sorted(vector[0], key=lambda x: x[1], reverse=True)
        # topic_num, prop_topic = row[0]
        topic_num, prop_topic = max(vector[0], key=lambda x: x[1])
        if topic_num in (0, 1, 3, 5, 6, 11) and prop_topic > .04:
            pot_match.append((doc_num, topic_num, prop_topic))
    return pot_match 
    '''
    sorted_list = sorted(pot_match, key=lambda x: x[-1], reverse=True)
    return sorted_list
    '''
    
    
def corpus_parser(corpus_list, corpus_dict, ldamodel):
    other_corpus = [corpus_dict.doc2bow(text) for text in vol_list]
    sorted_list = analyze_corpus_with_model(other_corpus, ldamodel)
    best_match = max(sorted_list, key=lambda x: x[-1])
    most_common_topic = Counter([x[1] for x in sorted_list]).most_common(1).pop()
    top_topic = max(get_topic_average(sorted_list), key=lambda x: x[-1])
    return best_match, most_common_topic, top_topic
    

def file_parser(feature_reader, corpus_dict, analyzed_dict, ldamodel):
    for vol in feature_reader.volumes():
        corpus_list = volume_parser(vol)
        best_match, most_common_topic, top_topic = corpus_parser(corpus_list, corpus_dict, ldamodel)
        analyzed_dict[vol.id] = Book(vol.id, Topic(*top_topic), BestMatch(*best_match), Topic(*most_common_topic))
    return analyzed_dict
        
        
    

In [ ]:
files = glob.glob('../data/test/*.bz2')
fr = FeatureReader(files)

In [ ]:
fr = FeatureReader(files)
analyzed_dict = file_parser(fr, corpus_dict, {}, lda_model)

In [ ]:
analyzed_dict

## Measuring Performance 

In [ ]:
# Timeit
import timeit
def wrapper(func, *args, **kwargs):
    def wrapped():
        return func(*args, **kwargs)
    return wrapped

wrapped = wrapper(file_parser, fr, corpus_dict, {}, lda_model)

In [ ]:
import cProfile 
cProfile.run('file_parser(fr, {}, lda_model)')

In [ ]:
timeit.timeit(wrapped, number=1)